## Deep Learning Models Experiments -  Premier Analysis on Azure Machine Learning
### LSTM and DAN 

In [1]:
import argparse
import os
import sklearn
import pandas as pd 
import numpy as np
from sklearn.metrics import f1_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from azureml.core import Run, Dataset, Environment,Experiment,ScriptRunConfig
from sklearn.preprocessing import LabelEncoder
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

In [2]:
from azureml.core import  Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
interactive_auth = InteractiveLoginAuthentication(tenant_id="9ce70869-60db-44fd-abe8-d2767077fc8f")

ws = Workspace.from_config()

In [53]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: cdh-azml-dev-mlw
Azure region: eastus
Subscription id: 320d8d57-c87c-4434-827f-59ee7d86687a
Resource group: csels-cdh-dev


In [3]:
# current working directory
path = os.getcwd()
print("Current Directory:", path)
  
# parent directory
parent = os.path.join(path, os.pardir)
  
# prints parent directory
print("\nParent Directory:", os.path.abspath(parent))

premier_path = os.path.abspath(parent)

Current Directory: c:\Users\wsn8\Code\premier_analysis\azure_ml

Parent Directory: c:\Users\wsn8\Code\premier_analysis


### Create Compute

In [4]:
clustername = 'StandardNC6'
is_new_cluster = False
try:
    aml_cluster_gpu = ComputeTarget(workspace = ws,name= clustername)
    print("Find the existing cluster")
except ComputeTargetException:
    print("Cluster not find - Creating cluster.....")
    is_new_cluster = True
    compute_config = AmlCompute.provisioning_configuration(vm_size='StandardNC6',
                                                           max_nodes=2)
    aml_cluster_gpu = ComputeTarget.create(ws, clustername, compute_config)

aml_cluster_gpu.wait_for_completion(show_output=True)

Find the existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Environment for deep model training

In [5]:
%%writefile conda_dependencies_model.yml

channels:
- anaconda
- default
dependencies:
- python=3.8
- pip:
  - azureml-defaults
  - matplotlib
  - pandas
  - argparse
  - joblib
  - scikit-learn
  - azureml-sdk
  - openpyxl
  - tensorflow
  - keras-tuner

Overwriting conda_dependencies_model.yml


In [6]:
premier_train_model_env = Environment.from_conda_specification(name='premier_train_model_env', file_path='conda_dependencies_model.yml')
# Specify a GPU base image
premier_train_model_env.docker.enabled = True
premier_train_model_env.docker.base_image = DEFAULT_CPU_IMAGE
premier_train_model_env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210220.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": true,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "premier_train_model_env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "de

#### Experiments : Use only first inpatient day's worth of features (--day_one)

In [7]:

mod_names = ['dan','lstm']
outcomes = ['icu','death','misa_pt']
#outcomes = ['misa_pt']
EPOCHS = 1
for outcome in outcomes:
    for mod in mod_names:
        estimator = ScriptRunConfig(source_directory='./training',
                            script='train_model.py',
                            compute_target=aml_cluster_gpu,
                            arguments=['--day_one','--outcome', outcome,'--model', mod,'--epochs',EPOCHS],
                            environment=premier_train_model_env)

        exp_name = f"Job-train-deepmodel-d1-{outcome}-{mod}"
        print("Submit Experiment:",exp_name)
        # Create experiment
        experiment = Experiment(workspace=ws, name = exp_name)
        run = experiment.submit(estimator)

Submit Experiment: Job-train-deepmodel-d1-icu-dan
Submit Experiment: Job-train-deepmodel-d1-icu-lstm
Submit Experiment: Job-train-deepmodel-d1-death-dan
Submit Experiment: Job-train-deepmodel-d1-death-lstm
Submit Experiment: Job-train-deepmodel-d1-misa_pt-dan
Submit Experiment: Job-train-deepmodel-d1-misa_pt-lstm


#### Experiments: Use all features in lookback period (--all_days)


In [8]:
mod_names = ['dan','lstm']
outcomes = ['icu','death','misa_pt']
#outcomes = ['misa_pt']
EPOCHS = 1
for outcome in outcomes:
    for mod in mod_names:
        estimator = ScriptRunConfig(source_directory='./training',
                            script='train_model.py',
                            compute_target=aml_cluster_gpu,
                            arguments=['--all_days','--outcome', outcome,'--model', mod,'--epochs',EPOCHS],
                            environment=premier_train_model_env)

        exp_name = f"Job-train-deepmodel-all_days-{outcome}-{mod}"
        print("Submit Experiment:",exp_name)
        # Create experiment
        experiment = Experiment(workspace=ws, name = exp_name)
        run = experiment.submit(estimator)

Submit Experiment: Job-train-deepmodel-all_days-icu-dan
Submit Experiment: Job-train-deepmodel-all_days-icu-lstm
Submit Experiment: Job-train-deepmodel-all_days-death-dan
Submit Experiment: Job-train-deepmodel-all_days-death-lstm
Submit Experiment: Job-train-deepmodel-all_days-misa_pt-dan
Submit Experiment: Job-train-deepmodel-all_days-misa_pt-lstm
